In [16]:
import requests
import re
import glob, os, shutil
import gzip
import subprocess
from bs4 import BeautifulSoup
import random

### Part 3.1 Create a List of Android Apps

In [59]:
def get_submap_xmls(sitemap):
    resp = requests.get(sitemap)
    soup = BeautifulSoup(resp.content, 'xml')
    url = soup.find_all('loc') 
    result = []
    for link in url:
        result += [link.get_text()]
    return result
#extract categories list
def category(link_lst):
    result = [] 
    for xml in link_lst:
        result += [re.search('(?<=sitemaps\/)(.*)(?=\-\d)|(?<=sitemaps\/)(.*)(?=\.xml)',xml).groups()[1]]
    return [i for i in result if i] 


#get all the gz files from each categories
def sample_from_cat(categories): 
    soups = []
    for c in categories:
        url = 'https://apkpure.com/sitemaps/{}.xml.gz'.format(c)
        try:
            r = requests.get(url)
        except:
            url = 'https://apkpure.com/sitemaps/{}.xml.gz'.format(c+'-1')
            r = requests.get(url)
        #decompress the gz file and parse xml
        data = gzip.decompress(r.content)
        soup = BeautifulSoup(data,features = 'lxml')
        soups.append(soup)
    return soups
def get_app_urls(sitemap,cat,number):
    xmls = get_xmls(sitemap)
    
    if cat == 'all':
        categories = category(xmls)
    elif type(cat) == int:
        categories = random.choices(category(xmls), k = cat)
    else:
        categories = cat
        
    soups = sample_from_cat(categories)
    apps = []
    for soup in soups:
        count = 0
        sp = soup.find_all(re.compile('loc')) 
        lst = [] 
        for i in sp:
            if re.match('<loc>', str(i)) and count < number:
                try:
                    lst += [re.search('(?<=<loc>)(https:\/\/apkpure.com\/.*?\/.*[a-zA-Z\d].*)(?=<\/loc>)', str(i)).group()] #find all urls storec in loc
                    count += 1
                except:
                    continue
        apps += lst
    return apps

In [89]:
apps = get_app_urls('https://apkpure.com/sitemap.xml', ['dating','education'], 20)

In [90]:
apps

['https://apkpure.com/ellochat-meet-strangers-nearby-chat-and-dating/com.bsapps.randomchat',
 'https://apkpure.com/hot-or-not-find-someone-right-now/com.hotornot.app',
 'https://apkpure.com/find-date-connect-with-your-best-match-by-zoosk/com.zoosk.zoosk',
 'https://apkpure.com/match-dating-chat-date-meet-someone-new/com.match.android.matchmobile',
 'https://apkpure.com/eharmony-online-dating-app/com.eharmony',
 'https://apkpure.com/okcupid-the-1-online-dating-app-for-great-dates/com.okcupid.okcupid',
 'https://apkpure.com/moco-chat-meet-people/com.jnj.mocospace.android',
 'https://apkpure.com/cmb-free-dating-app/com.coffeemeetsbagel',
 'https://apkpure.com/blendr-chat-flirt-meet/com.blendr.mobile',
 'https://apkpure.com/meet24-love-chat-singles/com.wildec.meet24',
 'https://apkpure.com/hooked-on-phonics/com.BigYellowTaxi.HookedPhonics',
 'https://apkpure.com/learn-spanish-free-for-beginners/com.gonliapps.learnspanish',
 'https://apkpure.com/preschool-educational-games/com.gurucools.pla

In [76]:
download_apk(apps[0], 'data')

In [80]:
completeName = os.path.join('data'+'/', apps[0].split('/')[-1]+".apk")

In [87]:
#os.chdir('data')
subprocess.run(['apktool', 'd', completeName])   

FileNotFoundError: [Errno 2] No such file or directory: 'apktool': 'apktool'

In [ ]:
clean_disk(outpath)

### 3.2 Download and Decompile apk Files

In [72]:
def process_apk(outpath,url):
    if not os.path.exists(outpath+'/'):
        os.mkdir(outpath+'/')
    for link in url:
        req = requests.get(link)
        soup_1 = BeautifulSoup(req.text)
        download_link = link + '/download?from=details'
        req_1 = requests.get(download_link)
        soup_2 = BeautifulSoup(req_1.text)
        try:
            download_link_file = soup_2.find('div',attrs = {"class":"fast-download-box fast-bottom"}).find('p',attrs = {'class':'down-click'}).find('a',href = True)['href']
        except:
            continue
        req_2 = requests.get(download_link_file)
        data = req_2.content
        completeName = os.path.join(outpath+'/', link.split('/')[-1]+".apk")         
        with open(completeName, 'wb') as fh:
            fh.write(data)
        subprocess.call(['cd', outpath])
        subprocess.call(['apktool.txt', 'd', completeName])    
    

In [75]:
def download_apk(app_link, outpath): #download the apk file
    app_name = app_link.split('/')[-1]
    
    #get download link
    download_link = app_link + '/download?from=details'
    r = requests.get(download_link)
    apkfile = r.content
    
    #download an save in outpath directory
    path = os.path.join(outpath +'/', app_name +".apk")
    with open(path, 'wb') as f:
        f.write(apkfile)

In [78]:
def decompile(app_link, outpath): #decompile apk files and remove .apk
    app_name = app_link.split('/')[-1]
    try:
        os.chdir('data')
    except:
        continue
    completeName = os.path.join(outpath+'/', app_link.split('/')[-1]+".apk")      
    subprocess.call(['apktool', 'd', app_name +".apk"])
    if os.path.exists(app_name +".apk"): #delete apkfiles
        os.remove(app_name +".apk")

In [10]:
def get_smali_code(app_urls, outpath): #download and decompile all application urls
    for url in app_urls:
        try:
            page = get_download_page(url)
            download_apk(page, url, outpath)
            decompile(url, outpath)
        except:
            continue

### 3.3 Organize Disk

In [65]:
#use for d, dirs, files in os.walk(root):
def clean_app_folder(app_path):
    subs = os.listdir(app_path)
    for s in subs:
        if s not in ['smali', 'AndroidManifest.xml']:
            path = app_path+'/'+s
            if os.path.isdir(path):
                shutil.rmtree(path)
            elif os.path.isfile(path):
                os.remove(path)

In [73]:
def clean_disk(outpath):
    apps = os.listdir(outpath)
    for app in apps:
        if os.path.isdir(app):
            clean_app_folder(outpath+'/'+app)